In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    # Sequential object
    model = Sequential()
    # 卷積層 1
    model.add(Conv2D(filters=64, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu', 
                     input_shape=(32,32,3)))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 2
    model.add(Conv2D(filters=128, 
                     kernel_size=3, 
                     padding='same', 
                     activation='relu'))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 3
    model.add(Conv2D(filters=256, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu'))
    #2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 扁平層
    model.add(Flatten())
    # 丟棄法
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 輸出層
    model.add(Dense(10, activation='softmax'))

    # 優化器為 Adam
    model.compile( loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])
    
    return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import History, LearningRateScheduler

def step_decay(epoch): 
    """
    以每1/5使學習率進行衰減
    Parameters: 
        epoch(int): 訓練週期數 
        Returns : 學習率
    """
    lrate = 0.001
    if epoch >= 50: lrate /= 5.0
    if epoch >= 100: lrate /= 5.0
    if epoch >= 150: lrate /= 5.0

    return lrate

def train_batchsize(model, data, batch_size, epochs, decay):
    """
    Parameters:
        model(obj) : Model object
        data(tuple) : 訓練資料、測試資料
        batch_size(int): 批次尺寸
        epochs(int) : 訓練週期數
        decay(float) : 學習率
    """
    x_train, y_train, x_test, y_test = data
    # 訓練資料
    train_gen = ImageDataGenerator(rescale=1.0/255.0,        # 正規化
                                   width_shift_range=0.1,    # 以 0.1的比例隨機橫向移動 
                                   height_shift_range=0.1,   # 以0.1的比例隨機上下移動 
                                   rotation_range=10,        # 在10度的範圍內隨機旋轉 
                                   zoom_range=0.1,           # 以原始尺寸的0.1比例隨機放大 
                                   horizontal_flip=True      # 左右翻轉
                                  ).flow(x_train, y_train, batch_size=batch_size)
    # 驗證資料
    test_gen = ImageDataGenerator(
        rescale=1.0/255.0                       # 正規化
        ).flow(x_test, y_test, batch_size=128)  # 批次大小為128  
    
    hist = History()

    model.fit(train_gen,
              steps_per_epoch=x_train.shape[0] // batch_size,
              epochs=epochs,
              validation_data=test_gen,
              callbacks=[hist, decay])

    return hist.history


In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def train(train_mode):
    """
    train_mode(int);
    # 0: normal batch_size=128,
        lr=0.001, 0.0002, 0.00004, 0.000008
    # 1: increase batch = 128, 640, 3200, 16000
        lr=0.001
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    data = (x_train, y_train, x_test, y_test)

    # 建立模型
    model = make_convlayer()

    # 維持History物件的清單
    histories = []
    
    # 學習率減衰排程
    decay = LearningRateScheduler(step_decay)
    
    # 批次大小排成
    same_lr = LearningRateScheduler(lambda epoch: 0.001)
    
    # 學習率減衰
    if train_mode == 0:
        histories.append(train_batchsize(
        model, data, batch_size=128, epochs=200, decay=decay))
    # 增加批次尺寸
    if train_mode == 1:
        histories.append(train_batchsize(model, data, batch_size=128, epochs=50, decay=same_lr))
        histories.append(train_batchsize(model, data, batch_size=640, epochs=50, decay=same_lr))
        histories.append(train_batchsize(model, data, batch_size=3200, epochs=50, decay=same_lr))
        histories.append(train_batchsize(model, data, batch_size=16000, epochs=50, decay=same_lr)) 
        
    # 整合History
    joined_history = histories[0]
    for i in range(1, len(histories)):
        for key, value in histories[i].items():
            joined_history[key] = joined_history[key] + value

    return joined_history


In [ ]:
%%time
history = train(0)

In [ ]:
%%time
history_batch = train(1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(15, 10)) # 繪圖尺寸 
plt.subplot(2, 1, 1) # 在2 × 1 格線的上方繪圖
# 繪製學習率衰減的精確度走勢
plt.plot(history['accuracy'], label='lr', linestyle='--')
# 繪製增加批次尺寸的精確度走勢
plt.plot(history_batch['accuracy'], label='batch')
plt.legend()        # 顯示圖例
plt.grid()          # 顯示格線
plt.xlabel('Epoch')   # x 軸標籤
plt.ylabel('Acc')     # y 軸標籤


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(15, 15)) # 繪圖尺寸 
plt.subplot(2, 1, 1) # 在2 × 1 格線的上方繪圖

# 繪製學習率衰減的精確度走勢
plt.plot(history['val_accuracy'], label='lr', linestyle='--')
# 繪製增加批次尺寸的準確度走勢
plt.plot(history_batch['val_accuracy'], label='batch')
plt.legend()           # 顯示圖例
plt.grid()             # 顯示格線
plt.xlabel('Epoch')    # x 軸標籤
plt.ylabel('Acc')      # y 軸標籤
